# Detect AI generated text using Naive Bayes Classification

Import required libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
training_data = pd.read_csv('train_essays.csv')
training_data

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
1383,86hfhd2,1,I am writing to express my perspective on the ...,1
1384,970wesgs,1,I hope this letter finds you well. I am writin...,1
1385,9831fsdga1,1,I trust this letter reaches you in good health...,1
1386,983495erg,1,I hope this letter reaches you in good health ...,1


In [3]:
prompt_data = pd.read_csv('train_prompts.csv')
prompt_data.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [4]:
testing_data = pd.read_csv('test_essays.csv')
testing_data.head()

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [23]:
### HYPER PARAMETERS ###
vocabulary = ['the','in','is','and','me','I','but','has','not']
min_freq_threshold = [20,10,10,10,5,6,5,5,3]

In [24]:
def word_probability(essay):
    words = nltk.word_tokenize(essay)
    frequency_distribution = nltk.FreqDist(words)
    probability_dict={}
    for word in vocabulary:
        probability_dict[word] = [frequency_distribution[word]]

    #tagged_words = nltk.pos_tag(words)
    #errors = [word for word, tag in tagged_words if tag == 'VBZ' or tag == 'VBP']
    ### return human error probability ###
    return probability_dict

In [25]:
def convert_essays_to_frequency(df):
    prob_df = pd.DataFrame(columns=vocabulary)
    
    training =(True if 'generated' in list(df.columns) else False)
    
    for index,row in df.iterrows():
        P = word_probability(row['text']) 
        if training: 
            P['generated'] = int(row['generated'])
        prob_df = pd.concat([prob_df,pd.DataFrame(P)],ignore_index=True)
    return prob_df

In [26]:
def get_probabilities(raw_data):
    df = convert_essays_to_frequency(raw_data)
    columns = list(df.columns)
    columns.remove('generated')
    probs = {}
    probs_llm = {}
    for i in columns:
        probs[i]=0
        probs_llm[i]=0
    for index,row in df.iterrows():
        for i in columns:
            if row[i] >=min_freq_threshold[columns.index(i)]:
                if i in probs.keys():
                    probs[i]+=1
                if not row['generated']:
                    if i in probs_llm.keys():
                        probs_llm[i]+=1
    P_llm = len(df[df['generated']==1])/len(df.index)
    for key,val in probs.items():
        probs[key] = val/len(df.index)
    for key,val in probs_llm.items():
        probs_llm[key] = val/len(df.index)
    return probs,probs_llm,P_llm

In [27]:
def NBC(raw_data):
    probs_indv,probs_llm,P_L = get_probabilities(raw_data)
    P_V = 1
    P_V_L = 1
    for key,val in probs_indv.items():
        P_V*=val
        P_V_L*=probs_llm[key]
    P = (P_V_L * P_L)/P_V
    #return Bayesian prob, individual probs product, individual prob dictionary, individual prob given generated product, individual prob given generated dictionary, prob of generated
    return P,P_V,probs_indv,P_V_L,probs_llm,P_L

In [28]:
print(NBC(training_data))

(0.00916839642922549, 4.3142550815283406e-08, {'the': 0.803314121037464, 'in': 0.4675792507204611, 'is': 0.43515850144092216, 'and': 0.6102305475504323, 'me': 0.0007204610951008645, 'I': 0.06484149855907781, 'but': 0.11311239193083573, 'has': 0.12896253602305474, 'not': 0.6347262247838616}, 4.2232356636417144e-08, {'the': 0.7989913544668588, 'in': 0.46397694524495675, 'is': 0.43515850144092216, 'and': 0.6066282420749279, 'me': 0.0007204610951008645, 'I': 0.06484149855907781, 'but': 0.11311239193083573, 'has': 0.12896253602305474, 'not': 0.6332853025936599}, 0.009365994236311239)
